In [1]:
import sys
!{sys.executable} -m pip install -q -q -r requirements.txt

In [2]:
import datetime, os, random, shutil, urllib.request, zipfile, time, warnings
warnings.filterwarnings('ignore')
from functools import wraps
from math import trunc
import numpy as np
import pandas as pd
from pathlib import Path
from urllib.request import urlopen
from zipfile import ZipFile
from scipy.sparse.linalg import norm
import scipy.sparse as ss
from scipy.sparse.linalg import svds
from tqdm import tqdm
from numba import njit
from sklearn.metrics import mean_squared_error as mse
import plotly.express as px

SEED = 123
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything()

# Preprocessing

In [3]:
def get_dataset():
    path = Path("m1.zip")
    if not path.exists():
        with path.open("wb") as f:
            print("Downloading dataset...")
            f.write(urlopen("http://files.grouplens.org/datasets/movielens/ml-1m.zip").read())
    if not Path("ml-1m").is_dir():
        print("unzipping...")
        with ZipFile("m1.zip") as zf:
            zf.extractall()
    ratings_list = [i.strip().split("::") for i in open('ml-1m/ratings.dat', 'r').readlines()]
    ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
    ratings_df['Rating'] = ratings_df['Rating'].apply(pd.to_numeric)
    return ratings_df

In [6]:
def split(df):
    R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
    seed_everything()
    test_indices = np.array(random.sample(list(np.argwhere(R_df.values>0)), 2500))

    x_indices = test_indices[:,0]
    y_indices = test_indices[:,1]
    u_ids = R_df.index[x_indices].astype(np.int32)
    i_ids = R_df.columns[y_indices].astype(np.int32)
    
    df['UserID'] = df['UserID'].astype(np.int32)
    df['MovieID'] = df['MovieID'].astype(np.int32)
    df['Rating'] = df['Rating'].astype(np.float64)
    
    df= df.drop(columns=['Timestamp'], errors='ignore')
    
    test = []
    for u_id, i_id in zip(u_ids, i_ids):
        test.append(df.loc[(df['UserID'] == u_id) & (df['MovieID'] == i_id)])
    test_df = pd.concat(test)
    train_df= df.drop(test_df.index.tolist())
    
    return train_df, test_df

In [7]:
ratings_df = get_dataset()
train_df, test_df = split(ratings_df)

# Funk-SVD

## Numba Methods

In [15]:
@njit
def _shuffle(X):
    np.random.shuffle(X)
    return X

@njit
def _initialization(n_users, n_items, n_factors):
    bu = np.zeros(n_users)
    bi = np.zeros(n_items)
    pu = np.random.normal(0, .1, (n_users, n_factors))
    qi = np.random.normal(0, .1, (n_items, n_factors))
    return bu, bi, pu, qi

@njit
def _run_epoch(X, bu, bi, pu, qi, global_mean, n_factors, lr, reg):
    for i in range(X.shape[0]):
        user, item, rating = int(X[i, 0]), int(X[i, 1]), X[i, 2]

        # Predict current rating
        pred = global_mean + bu[user] + bi[item]

        for factor in range(n_factors):
            pred += pu[user, factor] * qi[item, factor]
        err = rating - pred

        # Update biases
        bu[user] += lr * (err - reg * bu[user])
        bi[item] += lr * (err - reg * bi[item])

        # Update latent factors
        for factor in range(n_factors):
            puf = pu[user, factor]
            qif = qi[item, factor]
            pu[user, factor] += lr * (err * qif - reg * puf)
            qi[item, factor] += lr * (err * puf - reg * qif)
    return bu, bi, pu, qi

@njit
def _compute_metrics(X_val, bu, bi, pu, qi, global_mean, n_factors):
    residuals = []
    for i in range(X_val.shape[0]):
        user, item, rating = int(X_val[i, 0]), int(X_val[i, 1]), X_val[i, 2]
        pred = global_mean
        if user > -1:
            pred += bu[user]
        if item > -1:
            pred += bi[item]
        if (user > -1) and (item > -1):
            for factor in range(n_factors):
                pred += pu[user, factor] * qi[item, factor]
        residuals.append(rating - pred)
    residuals = np.array(residuals)
    loss = np.square(residuals).mean()
    rmse = np.sqrt(loss)
    mae = np.absolute(residuals).mean()
    return loss, rmse, mae

## Funk SVD Class

In [35]:
class SVD:
    def __init__(self, lr=.005, reg=.02, n_epochs=20, n_factors=100,
                 early_stopping=False, shuffle=False, min_delta=.001,
                 min_rating=1, max_rating=5):

        self.lr = lr
        self.reg = reg
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        self.early_stopping = early_stopping
        self.shuffle = shuffle
        self.min_delta = min_delta
        self.min_rating = min_rating
        self.max_rating = max_rating

    def fit(self, X, X_val=None):
        X = self._preprocess_data(X)

        if X_val is not None:
            X_val = self._preprocess_data(X_val, train=False, verbose=False)
            self._init_metrics()
        self.global_mean_ = np.mean(X[:, 2])
        return self._run_sgd(X, X_val)

    def _preprocess_data(self, X, train=True, verbose=True):
        X = X.copy()

        if train:  # Mappings have to be created
            user_ids = X['UserID'].unique().tolist()
            item_ids = X['MovieID'].unique().tolist()
            n_users = len(user_ids)
            n_items = len(item_ids)
            user_idx = range(n_users)
            item_idx = range(n_items)
            self.user_mapping_ = dict(zip(user_ids, user_idx))
            self.item_mapping_ = dict(zip(item_ids, item_idx))
        X['UserID'] = X['UserID'].map(self.user_mapping_)
        X['MovieID'] = X['MovieID'].map(self.item_mapping_)

        # Tag validation set unknown users/items with -1 (enables
        # `fast_methods._compute_val_metrics` detecting them)
        X.fillna(-1, inplace=True)
        
        X['UserID'] = X['UserID'].astype(np.int32)
        X['MovieID'] = X['MovieID'].astype(np.int32)
        return X[['UserID', 'MovieID', 'Rating']].values

    def _init_metrics(self):
        metrics = np.zeros((self.n_epochs, 3), dtype=np.float)
        self.metrics_ = pd.DataFrame(metrics, columns=['Loss', 'RMSE', 'MAE'])

    def _run_sgd(self, X, X_val):
        metric_names = 'Loss', 'RMSE', 'MAE'
        n_users, n_items = len(np.unique(X[:, 0])), len(np.unique(X[:, 1]))
        val_loss, val_rmse = '', ''
        bu, bi, pu, qi = _initialization(n_users, n_items, self.n_factors)

        # Run SGD
        pbar = tqdm(range(self.n_epochs), desc='Epoch', ncols=110)
        for epoch_ix in pbar:
            bu, bi, pu, qi = \
                _run_epoch(X, bu, bi, pu, qi, self.global_mean_, self.n_factors, self.lr, self.reg)
            
            if X_val is not None:
                self.metrics_.loc[epoch_ix, :] = _compute_metrics(
                                                     X_val, bu, bi, pu, qi,
                                                     self.global_mean_,
                                                     self.n_factors)
                
                f = lambda x: f"{self.metrics_.loc[epoch_ix, x]:.3f}"
                pbar.set_postfix({f'val_{m.lower()}': f(m) for m in metric_names})
                if self._early_stopping(epoch_ix, self.min_delta):
                    break

        self.bu_, self.bi_, self.pu_, self.qi_ = bu, bi, pu, qi
        return {f"val_{m.lower()}": self.metrics_[m].tolist() for m in metric_names}

    def predict(self, X, clip=True):
        return [self.predict_pair(u_id, i_id, clip)\
                for u_id, i_id in zip(X['UserID'], X['MovieID'])]

    def predict_pair(self, u_id, i_id, clip=True):
        user_known, item_known = False, False
        pred = self.global_mean_

        if u_id in self.user_mapping_:
            user_known = True
            u_ix = self.user_mapping_[u_id]
            pred += self.bu_[u_ix]
        if i_id in self.item_mapping_:
            item_known = True
            i_ix = self.item_mapping_[i_id]
            pred += self.bi_[i_ix]
        if user_known and item_known:
            pred += np.dot(self.pu_[u_ix], self.qi_[i_ix])
        if clip:
            pred = self.max_rating if pred > self.max_rating else pred
            pred = self.min_rating if pred < self.min_rating else pred
        return pred

    def _early_stopping(self, epoch_idx, min_delta):
        val_rmse = self.metrics_['RMSE'].tolist()
        if self.early_stopping and epoch_idx > 0:
            if val_rmse[epoch_idx] + min_delta > val_rmse[epoch_idx-1]:
                self.metrics_ = self.metrics_.loc[:(epoch_idx+1), :]
                return True
        return False

In [62]:
def rmse(y_true, y_pred):
    return mse(y_true, y_pred, squared=False)

def funk(train, test, **kw):
    hparams = '\n'.join(map(str, kw.items()))
    seed_everything()
    svd = SVD(**kw)
    hist = svd.fit(X=train, X_val=test)
    y_true, y_pred = test['Rating'], svd.predict(test)
    print(f"Test RMSE: {rmse(y_true, y_pred):.4f}\n\nHyperparams: \n{hparams}")
    return hist

hist = funk(train=train_df, test=test_df,
                shuffle=False, min_rating=1, max_rating=5, early_stopping=True,
                min_delta=.00001,
                lr=.0006,
                reg=.015,
                n_epochs=300,
                n_factors=25)

Epoch:  72%|████████████▏    | 215/300 [00:15<00:06, 14.15it/s, val_loss=0.732, val_rmse=0.856, val_mae=0.671]


Test RMSE: 0.8552

Hyperparams: 
('shuffle', False)
('min_rating', 1)
('max_rating', 5)
('early_stopping', True)
('min_delta', 1e-05)
('lr', 0.0006)
('reg', 0.015)
('n_epochs', 300)
('n_factors', 25)


In [63]:
px.line(pd.DataFrame(hist).rename_axis('Epoch'))\
    .update_layout(hoverlabel=dict(font_size=12, font_family="Rockwell"),
                   font=dict(family="Courier New, monospace", size=18))\
    .update_xaxes(showgrid=False, showspikes=True)


#     fig = px.line(data, x=hparam, y='score', facet_col='domain', 
#             line_dash='Lemmatized', color='Metric', line_shape='spline', hover_data={
#                 'Lemmatized': False,
#                 hparam: False,
#                 'domain': False,
#                 'Metric': True,
#                 'score': ":.3f"}).update_layout(title_text=f"Effect of '{hparam}' Value", title_x=0.5, hoverlabel=dict(
#                     font_size=12,
#                     font_family="Rockwell"),
#                     font=dict(family="Courier New, monospace", size=18))\
#             .update_traces(mode="markers+lines", hovertemplate="%{customdata[2]}=%{y:.3f}<extra></extra>")\
#             .update_xaxes(showgrid=False, showspikes=True)

In [9]:
# Training took 16 sec
# Test RMSE: 0.8416

# Hyperparams: 
# ('shuffle', False)
# ('min_rating', 1)
# ('max_rating', 5)
# ('early_stopping', False)
# ('min_delta', 1e-05)
# ('lr', 0.0007)
# ('reg', 0.02)
# ('n_epochs', 300)
# ('n_factors', 20)

######################
# Training took 20 sec
# Test RMSE: 0.8412

# Hyperparams: 
# ('shuffle', False)
# ('min_rating', 1)
# ('max_rating', 5)
# ('early_stopping', True)
# ('min_delta', 1e-05)
# ('lr', 0.0006)
# ('reg', 0.015)
# ('n_epochs', 1000)
# ('n_factors', 25)